In [174]:
## Toy dataset
train_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
    ['Yellow', 4, 'Lemon'],
]

In [175]:
columns = ['color', 'diameter', 'label']

In [176]:
def unique_vals(rows, col):
    '''
    rows: list of row data
    col: the index of column that we want to find
    '''
    return set([row[col] for row in rows])

In [177]:
unique_vals(train_data, 0)

{'Green', 'Red', 'Yellow'}

In [178]:
def class_count(rows):
    """
    Counts the number of each type of example in a dataset.
    rows: list of row data
    """
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [179]:
class_count(train_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 2}

In [180]:
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

In [181]:
print(is_numeric(7), is_numeric('Blue'))

True False


In [182]:
import numpy as np
from math import log2, e

In [183]:
def entropy(labels, base=None):
    """ Computes entropy of label distribution.
    Arguments:
    labels -- List of data's label    
    """

    n_labels = len(labels)

    if n_labels <= 1:
        return 0
    
    counts = class_count(labels)
    probs = [counts[key]/n_labels for key in counts.keys()]

    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0
    
    ent = 0

    for i in probs:
        ent -= i * log2(i)

    return ent

In [184]:
pure = [['Apple'], ['Apple']]
entropy(pure)

0

In [185]:
impure = [['Apple'], ['Orange']]
entropy(impure)

1.0

In [186]:
impure = [['Apple'], ['Orange'], ['Grape'], ['Grapefruit'], ['Blueberry']]
entropy(impure)

2.321928094887362

In [187]:
def info_gain(left, right, current_uncertainty):
    """ Information Gain.
    IG = The uncertainty of the starting node - the weighted impurity of two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))

    print("(1) Avg of Impurity = {:.4f} * {:.4f} + {:.4f} * {:.4f}".format(
        p, entropy(left), (1-p), entropy(right)))
    
    print("(2) Current uncertainty = {:.4f}".format(current_uncertainty))

    IG = current_uncertainty - (p * entropy(left) + (1-p) * entropy(right))

    print("(3) Information gain = {:.4f}".format(IG))

    return IG

In [188]:
current_uncertainty = entropy(train_data)
print("\nCurrent uncertainty ==> {:.4f}".format(current_uncertainty))


Current uncertainty ==> 1.5850


In [189]:
class Query:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def compare_with_query(self, example):
        '''
        Arguments:
        example -- List of row data (EX. ['Blue', 2, 'Blueberry'])
        '''
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is {} {} {}?".format(columns[self.column], condition, str(self.value))

In [190]:
Query(1, 3)

Is diameter >= 3?

In [191]:
Query(0, 'Green')

Is color == Green?

In [192]:
def partition(rows, query):
    """
    Partitions a dataset
    Arguments:
    rows -- List of row data
    query -- An object of Query class
    """
    true_rows, false_rows = [], []
    for row in rows:
        if query.compare_with_query(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [193]:
true_rows, false_rows = partition(train_data, Query(0, 'Red'))

print("The true_rows\n ===> ", true_rows, "\nThe false_rows\n ===> ", false_rows)

The true_rows
 ===>  [['Red', 1, 'Grape'], ['Red', 1, 'Grape']] 
The false_rows
 ===>  [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon'], ['Yellow', 4, 'Lemon']]


In [194]:
print('Query? ', Query(1,3))
true_rows, false_rows = partition(train_data, Query(1, 3))

print("The true_rows ===> ", true_rows, "\nThe false_rows ===> ", false_rows)

Query?  Is diameter >= 3?
The true_rows ===>  [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon'], ['Yellow', 4, 'Lemon']] 
The false_rows ===>  [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]


In [195]:
current_uncertainty = entropy(train_data)
true_rows, false_rows = partition(train_data, Query(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

(1) Avg of Impurity = 0.1667 * 0.0000 + 0.8333 * 1.5219
(2) Current uncertainty = 1.5850
(3) Information gain = 0.3167


0.31668908831502085

In [196]:
def find_best_split(rows):
    best_gain = 0
    best_query = None

    current_uncertainty = entropy(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])

        for val in values:
            query = Query(col,val)
            true_rows, false_rows = partition(rows, query)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            print('Qustion ====>>>> ', query)
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_query = gain, query

    return best_gain, best_query

In [197]:
best_gain, best_query = find_best_split(train_data)

print("\nThe best query ====>>>>>> ", best_query)

Qustion ====>>>>  Is color == Yellow?
(1) Avg of Impurity = 0.5000 * 0.9183 + 0.5000 * 0.9183
(2) Current uncertainty = 1.5850
(3) Information gain = 0.6667
Qustion ====>>>>  Is color == Red?
(1) Avg of Impurity = 0.3333 * 0.0000 + 0.6667 * 1.0000
(2) Current uncertainty = 1.5850
(3) Information gain = 0.9183
Qustion ====>>>>  Is color == Green?
(1) Avg of Impurity = 0.1667 * 0.0000 + 0.8333 * 1.5219
(2) Current uncertainty = 1.5850
(3) Information gain = 0.3167
Qustion ====>>>>  Is diameter >= 3?
(1) Avg of Impurity = 0.6667 * 1.0000 + 0.3333 * 0.0000
(2) Current uncertainty = 1.5850
(3) Information gain = 0.9183
Qustion ====>>>>  Is diameter >= 4?
(1) Avg of Impurity = 0.1667 * 0.0000 + 0.8333 * 1.5219
(2) Current uncertainty = 1.5850
(3) Information gain = 0.3167

The best query ====>>>>>>  Is diameter >= 3?


In [198]:
class Leaf:
    """ A Leaf node classifies data.
    
        This hlod a dictionaty of class (e.g., "Apple") -> number of times
        """
    
    def __init__(self, rows):
        self.predictions = class_count(rows)

In [199]:
class Decision_Node:
    """A Decision Node asks a query.
    
    This holds a reference to query, and to the two child nodes.
    """

    def __init__(self, query, true_branch, false_branch):
        self.query = query
        self.true_branch = true_branch
        self.false_branch = false_branch

In [200]:
def build_tree(rows):
    """Builds the tree.
    Argumnets:
    rows --- List of row data
    """
    gain, query = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, query)

    true_branch = build_tree(true_rows)

    false_rows = build_tree(false_rows)

    return Decision_Node(query, true_branch, false_rows)

In [201]:
def print_tree(node, spacing = ""):
    """Tree printing function."""

    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    print(spacing + str(node.query))

    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")

    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + " ")

In [202]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.query.compare_with_query(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [203]:
my_tree = build_tree(train_data)
print(type(my_tree))

Qustion ====>>>>  Is color == Yellow?
(1) Avg of Impurity = 0.5000 * 0.9183 + 0.5000 * 0.9183
(2) Current uncertainty = 1.5850
(3) Information gain = 0.6667
Qustion ====>>>>  Is color == Red?
(1) Avg of Impurity = 0.3333 * 0.0000 + 0.6667 * 1.0000
(2) Current uncertainty = 1.5850
(3) Information gain = 0.9183
Qustion ====>>>>  Is color == Green?
(1) Avg of Impurity = 0.1667 * 0.0000 + 0.8333 * 1.5219
(2) Current uncertainty = 1.5850
(3) Information gain = 0.3167
Qustion ====>>>>  Is diameter >= 3?
(1) Avg of Impurity = 0.6667 * 1.0000 + 0.3333 * 0.0000
(2) Current uncertainty = 1.5850
(3) Information gain = 0.9183
Qustion ====>>>>  Is diameter >= 4?
(1) Avg of Impurity = 0.1667 * 0.0000 + 0.8333 * 1.5219
(2) Current uncertainty = 1.5850
(3) Information gain = 0.3167
Qustion ====>>>>  Is color == Yellow?
(1) Avg of Impurity = 0.7500 * 0.9183 + 0.2500 * 0.0000
(2) Current uncertainty = 1.0000
(3) Information gain = 0.3113
Qustion ====>>>>  Is color == Green?
(1) Avg of Impurity = 0.2500 

In [204]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
 Is diameter >= 4?
 --> True:
  Predict {'Lemon': 1}
 --> False:
  Is color == Yellow?
  --> True:
   Predict {'Apple': 1, 'Lemon': 1}
  --> False:
   Predict {'Apple': 1}
--> False:
 Predict {'Grape': 2}


In [205]:
classify(train_data[0], my_tree)

{'Apple': 1}

In [210]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}

    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    
    return probs

In [211]:
print_leaf(classify(train_data[0], my_tree))

{'Apple': '100%'}